#                                     Analysis Pipeline

### Packages

In [1]:
print("Installing packages (this could take awhile)...")

import sys
!$sys.executable -m pip install  -r ../requirements.txt --quiet --exists-action i
print("Done!")

Installing packages (this could take awhile)...
Done!


In [2]:
import os, 
import glob
import re
import typing
import pathlib
import logging
import time
import warnings
import numpy
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from typing import List
module_path = os.path.abspath(os.path.join('../utils'))
if module_path not in sys.path:
    sys.path.append(module_path)
from analysis_func import *


  ##  Workflow

In [11]:
logging.basicConfig(
    format='%(asctime)s - %(name)-8s - %(levelname)-8s - %(message)s',
    datefmt='%d-%b-%y %H:%M:%S',
)
logger = logging.getLogger('Analysis workflow')
logger.setLevel(logging.INFO)


def analysis_worflow(inpDir:pathlib.Path, 
                     metaFile:pd.DataFrame, 
                     plate:str,
                     variableName:str,
                     outDir:pathlib.Path, 
                     dryrun:bool=True):
    if not dryrun:
        logger.info("Analysis workflow is running")
        path = pathlib.Path(inpDir, f'p{plate}_nyxus.csv')
        figDir = pathlib.Path(outDir, 'Figures-nyxus', f'plate{plate}')
        figDir.mkdir(parents=True, exist_ok=True)
        logger.info(f'Step 1: loading CSVs of plate: p{plate}')
        df = loading_csv(path)
        logger.info("CSVs are loaded")
        
        logger.info(f'Step 2: Metadata Processing')

        prf = metadata_processing(x=metaFile,
                                feat_data=df, 
                                plate=plate)
        
        logger.info(f'Step 3: Threshold estimation for cell size')

        outlier_thr = threshold_std(prf, variableName='AREA_PIXELS_COUNT', value=4) 
        outliers_small = [x for x in prf['AREA_PIXELS_COUNT'] if x < 15] 
        outliers_large= [x for x in prf['AREA_PIXELS_COUNT'] if x > outlier_thr] 
        outliers_removed= outliers_small + outliers_large
        outlier_p = float("{:.2f}".format(len(outliers_removed)/prf.shape[0] * 100))
        logger.info(f'Step 4: Plotting Figure 1: cellsize thresholding')
        plot_cellsize_threshold(prf, outlier_thr, outlier_p, plate, figDir)
        logger.info(f'Step 5: Data cleaning')
        dfclean =prf.query('AREA_PIXELS_COUNT > 15 and AREA_PIXELS_COUNT < @outlier_thr')
        
        logger.info(f'Step 6: Plotting Figure 2: thresholds_distributions_plots')
        
        thr_methods=[find_threshold, find_threshold, threshold_otsu, threshold_std]
        values=[0.001, 0.00001, 512, 4]
        otsus=[False, False, True, False]
        names =['FPR-1E-03', f'FPR-1E-05', 'OTSU', 'MEAN+4STD']

        
        thresholds_distributions_plots(dfclean,
                                  thr_methods=thr_methods,
                                  values=values,
                                  otsus=otsus,
                                  names=names,
                                  variableName='MEAN',
                                  figDir=figDir, 
                                  plate=plate
                                     )
            
        logger.info(f'Step 7: Calculating Covid expression proportion')

        

        prf = threshold_calculations(dfclean,
                                  variableName="MEAN",
                                  thr_methods=thr_methods,
                                  values=values,
                                  otsus=otsus)  


        varlist = prf.columns[1:].tolist()


        for v in varlist:
            
            if v == "CellCount":
                
                title=f'Mean Well Cell Count [{v}]'
                
            else:
                title=f'Mean Well Covid Reporter Expression [{v}]'
                
                

            heatmap_visualization(prf, 
                            variableName=v,
                            title=title,
                            plate=plate,
                            figDir=figDir
                      )


        filenames= (dfclean
                    .groupby(by = ['intensity_image', 'well_name'])
                    .count()
                    .reset_index()
                    .iloc[:, :2]

                   )

        prf = filenames.merge(prf, on='well_name')

        outcsv = pathlib.Path(outDir, 'overlay_csv')
        outcsv.mkdir(parents=True, exist_ok=True)
        
        logger.info(f'Step 8: Saving CSV outputs')

        prf.to_csv(f'{outcsv}/plate{plate}.csv', index=False)
        
        
        return 
    
    
    

## Inputs

In [15]:
platesNum=200
variableName="MEAN"
inpDir=pathlib.Path(pathlib.Path(os.getcwd()).parent.parent, 'outputs', 'nyxus')
outDir=pathlib.Path(pathlib.Path(os.getcwd()).parent.parent, 'outputs')
metaDir=pathlib.Path(pathlib.Path(os.getcwd()).parent.parent, 'metadata')


In [14]:
@my_timer
def run_analysis_workflow(platesNum:int,
                          variableName:str,
                          outDir:pathlib.Path,
                          metaDir:pathlib.Path):
    
    tplates = [str(i).zfill(3) for i in range(1, platesNum+1)]
    missing_plates = [str(i)for i in range(187, 191)]
    plates = [pl for pl in tplates if not pl in missing_plates]
    
    fname = [f for f in os.listdir(metaDir) if f.endswith('.csv')][0]
    
    metaFile = pd.read_csv(os.path.join(metaDir, 
                                        fname)
                          )
    
    for plate in plates:
           
        warnings.filterwarnings('ignore')
        os.environ["CUDA_VISIBLE_DEVICES"] = "0"

        starttime = time.time()
        
        figDir = pathlib.Path(outDir, 'figures_nyxus', f'p{plate}')
        
        prf= analysis_worflow(inpDir=inpDir, 
                              metaFile=metaFile,
                              plate=plate,
                              variableName=variableName,
                              outDir=outDir,
                              dryrun=False)

        endtime = round((starttime - time.time()) / 60, 3)

        logger.info(f'Total time to complete plate{plate} is {endtime} in minutes')
        
   
    

run_analysis_workflow(platesNum=platesNum,
                    variableName=variableName,
                    outDir=outDir,
                    metaDir=metaDir)
    

    
    
    